In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['No'].tolist()

总数量：1


[1]

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('start', '')
dict_param.pop('num', '')

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = url[:-1]
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))
        
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            count = int(html.xpath('//span[@id="lblResultCount"]/text()')[0].strip())
            if count > 0:
                with open('./page.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        continue

count

1


132

In [6]:
import math

df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Url': request_url,
                         'SKU Count': count,
                         'Page': math.ceil(count / 100)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Url,SKU Count,Page
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...,132,2


In [7]:
crawler_status = 'ok'

crawler_status

'ok'

In [8]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [9]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_page.xlsx', index=False)

output_correct

,No,Url,SKU Count,Page
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...,132,2


In [10]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_page_error.xlsx', index=False)

output_error

""
